In [ ]:
pip install pyarrow


In [ ]:
pip install fastparquet

In [1]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime


Consigna: def userdata(User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [ ]:
df_steam_games = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_steam_games.parquet"
df_steam_games = pd.read_parquet(df_steam_games )
df_steam_games

In [ ]:
df_steam_games.info()

In [5]:
df_steam_games_na = df_steam_games.dropna()

In [6]:
df_steam_games_na_dup = df_steam_games_na.drop_duplicates()


In [ ]:
# Cambio el nombre app_name por item_name para poder hacer merge con users_items

df_steam_games = df_steam_games.rename(columns={'app_name': 'item_name'})
df_steam_games.head()

In [8]:
aux_steam = df_steam_games

In [ ]:
aux_steam.head()


In [ ]:
# Solo necesito item_name y price

columnas_borrar_steam = ['publisher',	'genres',	'url',	'release_date',	'reviews_url',	'specs',	'early_access',	'id',	'developer',	'release_year']
aux_steam = aux_steam.drop(columnas_borrar_steam, axis=1)
aux_steam

In [ ]:
# Ubicación local archivo user_reviews https://github.com/MarceCorreal/PI1_MachineLearning/blob/main/Funciones/Developer/df_developer.parquet
df_user_reviews = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_user_reviews.parquet"
df_user_reviews = pd.read_parquet(df_user_reviews)
df_user_reviews

In [ ]:
df_user_reviews.head()

In [ ]:
# Solo necesito de este item_id y recommend

borrar_reviews = ['user_id',	'user_url',	'posted',	'sentiment_analysis']
aux_reviews = df_user_reviews.drop(borrar_reviews, axis=1)
aux_reviews

In [14]:
aux_reviews = aux_reviews.dropna()

In [15]:
aux_reviews_dup = aux_reviews.drop_duplicates()

In [16]:
aux_reviews = aux_reviews_dup

In [147]:
#Ahora user_items

In [ ]:
df_users_items = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_users_items.parquet"
df_users_items = pd.read_parquet(df_users_items)
df_users_items 

In [ ]:
# De user_items solo necesito user_id, item_id, item_name y playtimeforever

columnas_borrar_items = ['user_url', 'playtime_2weeks']
aux_items = df_users_items.drop(columnas_borrar_items, axis=1)
aux_items


In [150]:
# Hago merge entre aux_items y aux_reviews a travez de item_id

In [19]:
aux_items = aux_items.dropna()

In [20]:
aux_items = aux_items.drop_duplicates()


In [21]:

aux_reviews_items = pd.merge(aux_items, aux_reviews, on='item_id', how='inner')



In [ ]:
aux_reviews_items.head()

In [ ]:
aux_steam.head()

In [24]:
aux_steam.size

442322

In [25]:
# hacer merge aux_stem con aux_reviews_items a travez de item_name

dataset_endpoint_2 = pd.merge(aux_steam, aux_reviews_items, on= 'item_name', how='inner')

In [ ]:
dataset_endpoint_2.head()

In [27]:
dataset_endpoint_2.size

328003900

In [28]:
dataset_endpoint_2_nan = dataset_endpoint_2.dropna()

In [29]:
dataset_endpoint_2_nan_dup = dataset_endpoint_2_nan.drop_duplicates()

In [30]:
dataset_endpoint_2 = dataset_endpoint_2_nan_dup

In [31]:
dataset_endpoint_2.size
#No disminuyo nada

24601031

In [ ]:
dataset_endpoint_2.head()

In [33]:
# Creo columnas 'total_true' y 'total_false'

dataset_endpoint_2 = dataset_endpoint_2.assign(total_true=dataset_endpoint_2['recommend'].astype(int), total_false=dataset_endpoint_2['recommend'].astype(int))


In [ ]:
dataset_endpoint_2.head()

In [35]:
# Creo  columna 'total_recommend'
dataset_endpoint_2['total_recommend'] = dataset_endpoint_2['total_true'] + dataset_endpoint_2['total_false']

In [ ]:
dataset_endpoint_2.head()

In [37]:
dataset_endpoint_2['percentage_true'] = (dataset_endpoint_2['total_true'] / dataset_endpoint_2['total_recommend']) * 100

In [38]:
dataset_endpoint_2['percentage_true'] = dataset_endpoint_2['percentage_true'].astype(str) + '%'

In [ ]:
dataset_endpoint_2.head()

In [ ]:
# Se genera la columna cantidad total gastado

dataset_endpoint_2.loc[:, 'cantidad total gastado'] = dataset_endpoint_2['price'] * dataset_endpoint_2['items_count']

dataset_endpoint_2
#este df esta completo para el merge final

In [41]:
# nueva forma:

dataset_endpoint_2_grouped = dataset_endpoint_2.groupby('user_id').transform('sum')

dataset_endpoint_2 = pd.concat([dataset_endpoint_2, dataset_endpoint_2_grouped.add_suffix('_sum')], axis=1)



In [ ]:
dataset_endpoint_2.head()


In [119]:
columnas_borrar_final = [item_name	price	items_count	item_id	playtime_forever	recommend	total_true	total_false	total_recommend	price_sum	items_count_sum	playtime_forever_sum	recommend_sum	total_true_sum	total_false_sum	total_recommend_sum		cantidad total gastado_sum]

dataset_endpoint_2_grouped = dataset_endpoint_2_grouped.drop(columnas_borrar_final, axis=1)

dataset_endpoint_2_grouped

NameError: name 'dataset_endpoint_2_grouped' is not defined

In [99]:
dataset_endpoint_2 = dataset_endpoint_2_grouped

In [100]:
dataset_endpoint_2.to_parquet('dataset_endpoint_2.parquet')
